In [1]:
%pip install open3d scipy numpy

  Using cached open3d-0.19.0-cp312-cp312-win_amd64.whl.metadata (4.2 kB)
  Using cached dash-3.0.2-py3-none-any.whl.metadata (10 kB)
  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached importlib_metadata-8.6.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
Using cached open3d-0.19.0-cp312-cp312-win_amd64.whl (69.2 MB)
Using cached dash-3.0.2-py3-none-any.whl (7.9 MB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (

# 5.1

In [2]:
import open3d as o3d
import numpy as np
from scipy.stats import ortho_group

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Load 2 consecutive point clouds
pcd1 = o3d.io.read_point_cloud("selected_pcds\pointcloud_0000.pcd")
pcd2 = o3d.io.read_point_cloud("selected_pcds\pointcloud_0004.pcd")

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\p'
C:\Users\vikra\AppData\Local\Temp\ipykernel_24732\1690278179.py:2: SyntaxWarning: invalid escape sequence '\p'
  pcd1 = o3d.io.read_point_cloud("selected_pcds\pointcloud_0000.pcd")
C:\Users\vikra\AppData\Local\Temp\ipykernel_24732\1690278179.py:3: SyntaxWarning: invalid escape sequence '\p'
  pcd2 = o3d.io.read_point_cloud("selected_pcds\pointcloud_0004.pcd")


In [4]:
# Downsample for faster processing
pcd1_down = pcd1.voxel_down_sample(voxel_size=0.05)
pcd2_down = pcd2.voxel_down_sample(voxel_size=0.05)

# Estimate normals (important for ICP)
pcd1_down.estimate_normals()
pcd2_down.estimate_normals()

In [5]:
# Generate a random orthonormal rotation + small translation
R = ortho_group.rvs(dim=3)
t = np.array([[0.2], [0.1], [0.0]])  # small random translation
init_transform = np.eye(4)
init_transform[:3, :3] = R
init_transform[:3, 3:] = t

In [6]:

# Evaluate initial alignment
init_eval = o3d.pipelines.registration.evaluate_registration(
    pcd1_down, pcd2_down, 0.1, init_transform
)
print("Initial Alignment")
print("Fitness:", init_eval.fitness)
print("Inlier RMSE:", init_eval.inlier_rmse)

# Run ICP
reg_p2p = o3d.pipelines.registration.registration_icp(
    pcd1_down, pcd2_down, 0.1, init_transform,
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

print("\nAfter ICP")
print("Estimated Transformation:\n", reg_p2p.transformation)
print("Fitness:", reg_p2p.fitness)
print("Inlier RMSE:", reg_p2p.inlier_rmse)

Initial Alignment
Fitness: 0.020615384615384615
Inlier RMSE: 0.06462500634448524

After ICP
Estimated Transformation:
 [[-0.75299801  0.60898862  0.2492526   0.20567898]
 [ 0.57393527  0.42255565  0.70145921  0.03591155]
 [-0.32185758 -0.67125225  0.66770361 -0.08423332]
 [ 0.          0.          0.          1.        ]]
Fitness: 0.03169230769230769
Inlier RMSE: 0.0628131935229737


In [7]:
# Visualize alignment
pcd1_down.transform(reg_p2p.transformation)
o3d.visualization.draw_geometries([pcd1_down.paint_uniform_color([1, 0, 0]), 
                                   pcd2_down.paint_uniform_color([0, 1, 0])])

# 5.2

In [8]:
import open3d as o3d
import numpy as np
from scipy.stats import ortho_group
from copy import deepcopy

def run_icp(pcd1, pcd2, init_transform, threshold=0.1, method="point_to_point"):
    # Downsample
    pcd1_down = deepcopy(pcd1).voxel_down_sample(0.05)
    pcd2_down = deepcopy(pcd2).voxel_down_sample(0.05)
    
    # Estimate normals if needed
    pcd1_down.estimate_normals()
    pcd2_down.estimate_normals()

    # Choose method
    if method == "point_to_plane":
        estimation = o3d.pipelines.registration.TransformationEstimationPointToPlane()
    else:
        estimation = o3d.pipelines.registration.TransformationEstimationPointToPoint()

    # Initial evaluation
    init_eval = o3d.pipelines.registration.evaluate_registration(
        pcd1_down, pcd2_down, threshold, init_transform)
    
    # ICP
    reg_icp = o3d.pipelines.registration.registration_icp(
        pcd1_down, pcd2_down, threshold, init_transform, estimation)
    
    # Compute error between initial and learned transformation
    error = np.linalg.norm(reg_icp.transformation - init_transform)
    
    return {
        "fitness": reg_icp.fitness,
        "rmse": reg_icp.inlier_rmse,
        "error": error,
        "T": reg_icp.transformation,
    }

In [9]:
init_identity = np.eye(4)
init_random = np.eye(4)
init_random[:3, :3] = ortho_group.rvs(3)
init_random[:3, 3] = np.array([0.1, 0.1, 0.1])

# For RANSAC-based init:
def ransac_initial_guess(source, target):
    source_down = source.voxel_down_sample(0.05)
    target_down = target.voxel_down_sample(0.05)
    source_down.estimate_normals()
    target_down.estimate_normals()
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down,
        o3d.pipelines.registration.FPFHFeature(),
        o3d.pipelines.registration.FPFHFeature(),
        True, 0.075,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        4,
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9)],
        o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 100)
    )
    return result.transformation

# Run and collect results
experiments = [
    ("Identity", init_identity, "point_to_point", 0.1),
    ("Random", init_random, "point_to_point", 0.1),
    ("Random", init_random, "point_to_plane", 0.1),
    ("Identity", init_identity, "point_to_plane", 0.05),
]

results = []
for name, init, method, threshold in experiments:
    result = run_icp(pcd1, pcd2, init, threshold, method)
    results.append((name, method, threshold, result["fitness"], result["rmse"], result["error"]))

In [10]:
print(f"{'Init Guess':<12} | {'Method':<15} | {'Thresh':<7} | {'Fitness':<8} | {'RMSE':<8} | {'T Error':<8}")
print("-" * 70)
for name, method, threshold, fitness, rmse, error in results:
    print(f"{name:<12} | {method:<15} | {threshold:<7.3f} | {fitness:<8.4f} | {rmse:<8.4f} | {error:<8.4f}")

Init Guess   | Method          | Thresh  | Fitness  | RMSE     | T Error 
----------------------------------------------------------------------
Identity     | point_to_point  | 0.100   | 0.9937   | 0.0206   | 0.0044  
Random       | point_to_point  | 0.100   | 0.0602   | 0.0657   | 0.6188  
Random       | point_to_plane  | 0.100   | 0.0735   | 0.0656   | 0.8670  
Identity     | point_to_plane  | 0.050   | 0.9674   | 0.0179   | 0.0005  


In [11]:
print(result["T"])


[[ 9.99999995e-01 -5.12637423e-05 -8.79082824e-05 -4.56397355e-04]
 [ 5.12554576e-05  9.99999994e-01 -9.42423543e-05  2.32611106e-05]
 [ 8.79131131e-05  9.42378481e-05  9.99999992e-01 -6.31491481e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
